In [1]:
# !pip install -r requirements.txt

In [2]:
import torch
from torchvision.transforms.functional import to_pil_image
from ultralytics import YOLO
from os import path
import numpy as np
import cv2

from IPython.display import Image # to display images in the notebook

path_config_file = path.join('model_attempt.yaml')

In [3]:
model = YOLO(model='model_QR_detection.pt') # load in the finetuned model


In [4]:
def plot_bboxes(results):
    img = results[0].orig_img # original image
    names = results[0].names # class names dict
    scores = results[0].boxes.conf.cpu().numpy() # probabilities
    classes = results[0].boxes.cls.cpu().numpy() # predicted classes
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(np.int32) # bboxes
    for score, cls, bbox in zip(scores, classes, boxes): # loop over all bboxes
        class_label = names[cls] # class name
        label = f"{class_label} : {score:0.2f}" # bbox label
        lbl_margin = 3 #label margin
        img = cv2.rectangle(img, (bbox[0], bbox[1]),
                            (bbox[2], bbox[3]),
                            color=(0, 0, 255),
                            thickness=1)
        label_size = cv2.getTextSize(label, # labelsize in pixels 
                                     fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                                     fontScale=1, thickness=1)
        lbl_w, lbl_h = label_size[0] # label w and h
        lbl_w += 2* lbl_margin # add margins on both sides
        lbl_h += 2*lbl_margin
        img = cv2.rectangle(img, (bbox[0], bbox[1]), # plot label background
                             (bbox[0]+lbl_w, bbox[1]-lbl_h),
                             color=(0, 0, 255), 
                             thickness=-1) # thickness=-1 means filled rectangle
        cv2.putText(img, label, (bbox[0]+ lbl_margin, bbox[1]-lbl_margin), # write label to the image
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1.0, color=(255, 255, 255 ),
                    thickness=1)
    return img

In [5]:
# Holy moly everything below created by GPT

import cv2
import tkinter as tk
import random
from PIL import Image, ImageTk
import math

class App:
    def __init__(self, window):
        self.window = window
        self.window.title("Camera + Game")

        # --- Layout Frames ---
        self.left_frame = tk.Frame(window)
        self.left_frame.pack(side=tk.LEFT)

        # self.right_frame = tk.Frame(window)
        # self.right_frame.pack(side=tk.RIGHT)

        # --- Camera Setup ---
        self.cap = cv2.VideoCapture(0)
        # self.out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))

        self.video_label = tk.Label(self.left_frame)
        self.video_label.pack()
        self.update_camera()

        # --- Quit Button ---
        self.quit_button = tk.Button(self.left_frame, text="Quit", command=self.quit)
        self.quit_button.pack(pady=10)

    def update_camera(self):
        ret, frame = self.cap.read()
        if ret:
            # self.out.write(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)

            results = model(img)
            img_results = plot_bboxes(results=results)
            img_results_PIL = to_pil_image(img_results)

            imgtk = ImageTk.PhotoImage(image=img_results_PIL)
            # imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
        self.window.after(10, self.update_camera)

    def quit(self):
        self.cap.release()
        # self.out.release()
        self.window.destroy()

In [ ]:
root = tk.Tk()
app = App(root)
root.mainloop()